<a href="https://colab.research.google.com/github/Parinda29/MIR-Analysis/blob/main/MlProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from wordcloud import WordCloud
import re


In [21]:
#Upload Dataset
from google.colab import files
uploaded = files.upload()


Saving Musical_instruments_reviews2 (1).csv to Musical_instruments_reviews2 (1) (2).csv


In [22]:
#Checking dataset
!ls


'Musical_instruments_reviews2 (1) (1).csv'
'Musical_instruments_reviews2 (1) (2).csv'
'Musical_instruments_reviews2 (1).csv'
 sample_data


In [23]:
import pandas as pd

# Get the uploaded file name automatically
file_name = list(uploaded.keys())[0]

df = pd.read_csv(file_name)
df.head()


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,division
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014",positive
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013",positive
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013",positive
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014",positive
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014",positive


In [24]:
#Checking nunique data
df['division'].nunique()

3

In [25]:
#Checking unique data
df['division'].unique()

array(['positive', 'neutral', 'negative'], dtype=object)

In [26]:
#Counting Values
df['division'].value_counts()

,count
division,
positive,5898
neutral,772
negative,467


In [27]:
# Lowercase, remove punctuation, extra spaces
cleaned_texts = []
for text in df['reviewText']:
    text = str(text).lower()  # lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # remove non alphabetic chars
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces
    cleaned_texts.append(text)

# Adding cleaned text to dataframe
df['clean_text'] = cleaned_texts

# View first rows
df[['reviewText','clean_text']].head()


,reviewText,clean_text
0,"Not much to write about here, but it does exac...",not much to write about here but it does exact...
1,The product does exactly as it should and is q...,the product does exactly as it should and is q...
2,The primary job of this device is to block the...,the primary job of this device is to block the...
3,Nice windscreen protects my MXL mic and preven...,nice windscreen protects my mxl mic and preven...
4,This pop filter is great. It looks and perform...,this pop filter is great it looks and performs...
